In [1]:
import pandas as pd
import numpy as np
from utility_functions import load_file, pickle_file, starting_run, finished_run

In [2]:
codes = load_file("fully_filtered_codes.pickle")
summary_enhanced = load_file("summary_enhanced.pickle")
sid_costs = load_file("sid_costs.pickle")

In [3]:
rvu_lookup = pd.read_csv(
    "../../raw_data/PPRRVU18_OCT.csv", skiprows=9, index_col=0
)
conv_factor=rvu_lookup.at["47563", "FACTOR"]
rvu_lookup = rvu_lookup[["DESCRIPTION", "TOTAL.1"]].dropna()
summary_enhanced["CPT Costs"] = codes.query("cpt_flag == True")\
.join(rvu_lookup, on="codes", how="left")["TOTAL.1"].reset_index()\
.groupby("visit_link").sum().mul(conv_factor)
summary_enhanced["CPT Costs"].fillna(0, inplace=True)

In [4]:
summary_enhanced.index = summary_enhanced.index.astype('int')
summary_enhanced["SID_costs"] = sid_costs.groupby('visit_link').sum()
summary_enhanced["SID_costs"].fillna(0, inplace=True)
summary_enhanced["Cost"] = summary_enhanced["SID_costs"].add(summary_enhanced["CPT Costs"])

In [5]:
# Get rows that don't have an associated cost
missing_costs = sid_costs.set_index('visit_link')[sid_costs.set_index('visit_link')['SID_costs'].isna()].index
# Get rows that have a zero cost
zero_cost = summary_enhanced.query('Cost == 0').index
# If both conditions met, drop from summary table since it's a false zero
drop_rows = [index for index in missing_costs if index in zero_cost]
print(f"Number of rows dropped: {len(drop_rows)}")
summary_enhanced.drop(drop_rows, inplace=True)

Number of rows dropped: 93


In [6]:
pickle_file("summary_costs_enhanced.pickle", summary_enhanced)